# Transformers for Data Flow Graphs

In [1]:
import copy
import time
import math
import random
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from datasets import load_dataset

/home/jachym/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [6]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [ ]:
def split_data(data):
    train = []
    val = []
    test = []

    for el in data:
        pos = random.randrange(100)
        if (pos < 15):
            test.append(el)
        elif (pos >= 15 and pos < 30):
            val.append(el)
        else:
            train.append(el)
    return train, val, test

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [20]:
def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        if seq_len != bptt:  # only on last batch
            src_mask = src_mask[:seq_len, :seq_len]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            print("Data is:")
            print(data)
            print("Targets are:")
            print(targets)
            seq_len = data.size(0)
            if seq_len != bptt:
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

### Preprocessing data

1. Load data
2. Apply tokenizer
3. Build a vocabulary
4. Convert to a tensor
5. Put the data into batches

In [21]:
DATA_LOCATION = './data/data3node.txt'
train_iter = open(DATA_LOCATION, 'r').read().splitlines()
tokenizer = get_tokenizer(None)
batch_size = 20
eval_batch_size = 4

vocab = build_vocab_from_iterator(map(tokenizer,train_iter))

train_i, val_i, test_i = split_data(train_iter)
train_data = data_process(train_i)
val_data = data_process(val_i)
test_data = data_process(test_i)


train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bptt = 4

ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

best_val_loss = float('inf')
epochs = 3
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| epoch   1 |   200/ 1505 batches | lr 5.00 | ms/batch 27.65 | loss  5.46 | ppl   236.00
| epoch   1 |   400/ 1505 batches | lr 5.00 | ms/batch 27.00 | loss  6.15 | ppl   468.37
| epoch   1 |   600/ 1505 batches | lr 5.00 | ms/batch 26.30 | loss  6.11 | ppl   450.91
| epoch   1 |   800/ 1505 batches | lr 5.00 | ms/batch 28.40 | loss  6.12 | ppl   455.85
| epoch   1 |  1000/ 1505 batches | lr 5.00 | ms/batch 26.64 | loss  5.97 | ppl   390.45
| epoch   1 |  1200/ 1505 batches | lr 5.00 | ms/batch 28.77 | loss  6.00 | ppl   403.88
| epoch   1 |  1400/ 1505 batches | lr 5.00 | ms/batch 39.02 | loss  5.85 | ppl   346.54
Data is:
tensor([[7607, 3156,  255, 1390],
        [8047, 3156,  255, 1390],
        [7607, 3156,  256, 1390],
        [  21, 3157,  256, 1392]])
Targets are:
tensor([8047, 3156,  255, 1390, 7607, 3156,  256, 1390,   21, 3157,  256, 1392,
        4092, 3157,  256, 1392])
Data is:
tensor([[4092, 3157,  256, 1392],
        [  27, 3157,  257, 1393],
        [4092, 3157,  257, 1

Data is:
tensor([[4192, 7908,  329, 5156],
        [   5, 7908,  329, 5156],
        [ 190, 7908,  333, 5156],
        [  11, 7908,  333, 1443]])
Targets are:
tensor([   5, 7908,  329, 5156,  190, 7908,  333, 5156,   11, 7908,  333, 1443,
         190, 7909,  333, 1443])
Data is:
tensor([[ 190, 7909,  333, 1443],
        [  11, 7909,  333, 1443],
        [ 190, 3013,  334, 1443],
        [   0, 3013,  334, 5157]])
Targets are:
tensor([  11, 7909,  333, 1443,  190, 3013,  334, 1443,    0, 3013,  334, 5157,
        4185, 3013,  334, 5157])
Data is:
tensor([[4185, 3013,  334, 5157],
        [   1, 3013,  334, 5157],
        [4185, 3013,  335, 5158],
        [   4, 3013,  335, 5158]])
Targets are:
tensor([   1, 3013,  334, 5157, 4185, 3013,  335, 5158,    4, 3013,  335, 5158,
        4185, 3013, 4308, 1445])
Data is:
tensor([[4185, 3013, 4308, 1445],
        [  17, 7910, 4308, 1445],
        [4185, 7910, 4308, 1445],
        [   9, 3014, 4308, 5159]])
Targets are:
tensor([  17, 7910, 4308,

tensor([[4216, 6905,  381, 1664],
        [   4, 6906,  486, 1664],
        [4216, 6906,  486, 1664],
        [  21, 6906, 4393, 1664]])
Targets are:
tensor([   4, 6906,  486, 1664, 4216, 6906,  486, 1664,   21, 6906, 4393, 1664,
        4216, 6906, 4393, 1665])
Data is:
tensor([[4216, 6906, 4393, 1665],
        [  11, 6906,  494, 1665],
        [4217, 6906,  494, 1665],
        [  11, 3060,  494, 1666]])
Targets are:
tensor([  11, 6906,  494, 1665, 4217, 6906,  494, 1665,   11, 3060,  494, 1666,
        4217, 3060, 4394, 1666])
Data is:
tensor([[4217, 3060, 4394, 1666],
        [   4, 3060, 4394, 1666],
        [4217, 3060, 4394, 1666],
        [  16, 3060,  495, 1666]])
Targets are:
tensor([   4, 3060, 4394, 1666, 4217, 3060, 4394, 1666,   16, 3060,  495, 1666,
        4217, 3061,  495, 1667])
Data is:
tensor([[4217, 3061,  495, 1667],
        [  16, 3061,  495, 1667],
        [4217, 3061,  495, 1668],
        [   4, 3061, 7659, 1668]])
Targets are:
tensor([  16, 3061,  495, 1667, 42

Data is:
tensor([[ 225, 3196,  645, 5422],
        [  13, 3197, 7674, 5422],
        [ 225, 3197, 7674, 5423],
        [   8, 3197, 7675, 5423]])
Targets are:
tensor([  13, 3197, 7674, 5422,  225, 3197, 7674, 5423,    8, 3197, 7675, 5423,
         226, 3198, 7675, 5423])
Data is:
tensor([[ 226, 3198, 7675, 5423],
        [   7, 3198, 7676, 5423],
        [ 226, 3198, 7676, 5424],
        [   9, 3198, 7676, 5424]])
Targets are:
tensor([   7, 3198, 7676, 5423,  226, 3198, 7676, 5424,    9, 3198, 7676, 5424,
         226, 3198,  658, 5424])
Data is:
tensor([[ 226, 3198,  658, 5424],
        [  10, 3199,  658, 5425],
        [ 226, 3199,  658, 5425],
        [   0, 3199,  658, 5425]])
Targets are:
tensor([  10, 3199,  658, 5425,  226, 3199,  658, 5425,    0, 3199,  658, 5425,
         227, 3199,  659, 5426])
Data is:
tensor([[ 227, 3199,  659, 5426],
        [   1, 3200,  659, 5426],
        [ 227, 3200,  659, 5427],
        [   3, 3200, 7677, 5427]])
Targets are:
tensor([   1, 3200,  659,

tensor([   9, 3398,  503, 1623,   60, 3398,  503, 1623,    8, 3398,  503, 5292,
          60, 3399,  511, 5292])
Data is:
tensor([[  60, 3399,  511, 5292],
        [  35, 3399,  511, 5292],
        [  60, 7131, 4407, 1625],
        [   4, 7132, 4407, 1625]])
Targets are:
tensor([  35, 3399,  511, 5292,   60, 7131, 4407, 1625,    4, 7132, 4407, 1625,
          60, 7132, 4407, 1625])
Data is:
tensor([[  60, 7132, 4407, 1625],
        [  30, 7132, 4407, 1625],
        [  60, 7133,  519, 5293],
        [   4, 7133,  519, 5294]])
Targets are:
tensor([  30, 7132, 4407, 1625,   60, 7133,  519, 5293,    4, 7133,  519, 5294,
        4107, 7133,  519, 5294])
Data is:
tensor([[4107, 7133,  519, 5294],
        [  19, 7943,  519, 5294],
        [4107, 7135,  519, 5295],
        [   2, 7136, 4408, 5295]])
Targets are:
tensor([  19, 7943,  519, 5294, 4107, 7135,  519, 5295,    2, 7136, 4408, 5295,
        4107, 3411, 4408, 5295])
Data is:
tensor([[4107, 3411, 4408, 5295],
        [  26, 3412, 4408, 5

...

Data is:
tensor([[2913, 4076, 1164, 6048],
        [2913, 4076, 1164, 6048],
        [2913, 4076, 4916, 2101],
        [2914, 7604, 4917, 6055]])
Targets are:
tensor([2913, 4076, 1164, 6048, 2913, 4076, 4916, 2101, 2914, 7604, 4917, 6055,
        2914, 7604, 4917, 6055])
Data is:
tensor([[2914, 7604, 4917, 6055],
        [2914, 7604, 4917, 6055],
        [2915, 7604, 4917, 2102],
        [2915, 4083, 4918, 2102]])
Targets are:
tensor([2914, 7604, 4917, 6055, 2915, 7604, 4917, 2102, 2915, 4083, 4918, 2102,
        2915, 4083, 4918, 2102])
Data is:
tensor([[2915, 4083, 4918, 2102],
        [2915, 4083, 4919, 6056],
        [2916, 7605, 4919, 6056],
        [2916, 4084, 4919, 6056]])
Targets are:
tensor([2915, 4083, 4919, 6056, 2916, 7605, 4919, 6056, 2916, 4084, 4919, 6056,
        2916, 4084, 4919, 2103])
Data is:
tensor([[2916, 4084, 4919, 2103],
        [2916, 4084, 4920, 2103],
        [2916, 4084, 4920, 6057],
        [2916, 7606, 4920, 6057]])
Targets are:
tensor([2916, 4084, 4920,

Data is:
tensor([[6850, 8021, 4957, 6497],
        [6850, 8021, 4958, 6498],
        [6850, 3915, 4958, 6498],
        [6851, 3915, 4958, 6498]])
Targets are:
tensor([6850, 8021, 4958, 6498, 6850, 3915, 4958, 6498, 6851, 3915, 4958, 6498,
        6851, 7480, 1211, 6498])
Data is:
tensor([[6851, 7480, 1211, 6498],
        [6851, 7481, 1211, 6498],
        [6851, 7481, 1211, 6499],
        [6852, 7481, 1211, 6500]])
Targets are:
tensor([6851, 7481, 1211, 6498, 6851, 7481, 1211, 6499, 6852, 7481, 1211, 6500,
        6852, 7481, 1213, 6500])
Data is:
tensor([[6852, 7481, 1213, 6500],
        [6852, 3916, 1213, 6500],
        [6852, 3916, 1213, 2477],
        [6852, 7482, 1213, 2477]])
Targets are:
tensor([6852, 3916, 1213, 6500, 6852, 3916, 1213, 2477, 6852, 7482, 1213, 2477,
        6852, 7482, 1213, 2477])
Data is:
tensor([[6852, 7482, 1213, 2477],
        [6853, 7482, 1213, 2477],
        [6853, 7482, 4972, 2477],
        [6853, 3917, 4972, 2482]])
Targets are:
tensor([6853, 7482, 1213,

Data is:
tensor([[2812, 3966, 1463, 6542],
        [2812, 3966, 1463, 6543],
        [2813, 3966, 1464, 6543],
        [2813, 3966, 1464, 6543]])
Targets are:
tensor([2812, 3966, 1463, 6543, 2813, 3966, 1464, 6543, 2813, 3966, 1464, 6543,
        2814, 7533, 1464, 2533])
Data is:
tensor([[2814, 7533, 1464, 2533],
        [2814, 7534, 1464, 2533],
        [2815, 7535, 1464, 2533],
        [2815, 7535, 1465, 2535]])
Targets are:
tensor([2814, 7534, 1464, 2533, 2815, 7535, 1464, 2533, 2815, 7535, 1465, 2535,
        2816, 7535, 1465, 2535])
Data is:
tensor([[2816, 7535, 1465, 2535],
        [2816, 7535, 1466, 2535],
        [2805, 3979, 1466, 2535],
        [2805, 3979, 1466, 2535]])
Targets are:
tensor([2816, 7535, 1466, 2535, 2805, 3979, 1466, 2535, 2805, 3979, 1466, 2535,
        2805, 7543, 1466, 2536])
Data is:
tensor([[2805, 7543, 1466, 2536],
        [2805, 7543, 1466, 2536],
        [2806, 7543, 1466, 2536],
        [2806, 3980, 1451, 2536]])
Targets are:
tensor([2805, 7543, 1466,

Data is:
tensor([[2876, 4365, 5196, 6387],
        [2877, 4365, 5196, 6387],
        [2877, 4365, 1546, 6387],
        [2877, 4365, 5197, 6388]])
Targets are:
tensor([2877, 4365, 5196, 6387, 2877, 4365, 1546, 6387, 2877, 4365, 5197, 6388,
        2877, 4366, 5197, 6389])
Data is:
tensor([[2877, 4366, 5197, 6389],
        [2877, 4366, 5197, 6389],
        [2878, 4367, 5197, 6389],
        [2878, 4367, 5197, 6389]])
Targets are:
tensor([2877, 4366, 5197, 6389, 2878, 4367, 5197, 6389, 2878, 4367, 5197, 6389,
        2879, 4367, 1547, 6390])
Data is:
tensor([[2879, 4367, 1547, 6390],
        [2868, 4367, 1547, 6390],
        [2868,  414, 1547, 6390],
        [2868,  414, 1547, 6390]])
Targets are:
tensor([2868, 4367, 1547, 6390, 2868,  414, 1547, 6390, 2868,  414, 1547, 6390,
        2869,  414, 5198, 6390])
Data is:
tensor([[2869,  414, 5198, 6390],
        [2870,  414, 5198, 6374],
        [6765, 7647, 5198, 6375],
        [6765, 7647, 1548, 6375]])
Targets are:
tensor([2870,  414, 5198,

Data is:
tensor([[3107,  457, 1354, 6444],
        [3107,  458, 1354, 6444],
        [3107,  458, 1354, 6444],
        [3122,  459, 5088, 6444]])
Targets are:
tensor([3107,  458, 1354, 6444, 3107,  458, 1354, 6444, 3122,  459, 5088, 6444,
        3122,  459, 5088, 6444])
Data is:
tensor([[3122,  459, 5088, 6444],
        [6959,  459, 5088, 6445],
        [6959,  459, 5088, 6445],
        [6959,  459, 5088, 6445]])
Targets are:
tensor([6959,  459, 5088, 6445, 6959,  459, 5088, 6445, 6959,  459, 5088, 6445,
        6959,  459, 5089, 6446])
Data is:
tensor([[6959,  459, 5089, 6446],
        [3128,  459, 5089, 6446],
        [3128,  460, 5089, 6446],
        [3128,  460, 5089, 6447]])
Targets are:
tensor([3128,  459, 5089, 6446, 3128,  460, 5089, 6446, 3128,  460, 5089, 6447,
        3128,  461, 5089, 6447])
Data is:
tensor([[3128,  461, 5089, 6447],
        [3128,  461, 1355, 6447],
        [6961,  462, 5090, 6448],
        [6961,  462, 5090, 6448]])
Targets are:
tensor([3128,  461, 1355,

In [44]:
test_mask = generate_square_subsequent_mask(4).to(device)
test_input = torch.tensor([7607, 3156,  255, 1390])

# test_inputtensor([8047, 3156,  255, 1390, 7607, 3156,  256, 1390,   21, 3157,  256, 1392,
#         4092, 3157,  256, 1392])
best_model.eval()
out = best_model(test_input, test_mask)
pred = out.topk(1)[1].view(-1)[-1].item()
print(pred)

5822


#### Sources:

https://pytorch.org/tutorials/beginner/transformer_tutorial.html#define-the-model